# Random Forests

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Random subset of features

The $B$ bootstrapped dataset can be correlated.

A **random forest** is constructed by bagging + only a random" subset of $q \leq n$ features

Rule of thumb: $q = \sqrt{n}$ for classification trees and $q = \frac{n}{3}$ for regression trees

## 1. Scratch

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                test_size=0.3, shuffle=True, random_state=42)